In [1]:
from pyserini.search.lucene import LuceneSearcher
from datasets import load_dataset
from dotenv import load_dotenv
import os
from langchain.chat_models import ChatOpenAI, ChatCohere, ChatAnyscale

from main import get_query_expansion_dataset, run_search

/Users/jantiegges/.pyenv/versions/3.10.13/envs/550-final/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# set up LLMs, assumes that API keys are stored in .env file
load_dotenv(override=True)
chats = {}

# OpenAI
try: 
    openai_api_key = os.getenv("OPENAI_API_KEY")
    chat_openai = ChatOpenAI(openai_api_key=openai_api_key, model="gpt-3.5-turbo")
    chats["OpenAI"] = chat_openai
except:
    print("Could not initialize OpenAI chatbot. Please store a valid API key in a .env file.")

# Cohere
try:
    cohere_api_key = os.getenv("COHERE_API_KEY")
    chat_cohere = ChatCohere(cohere_api_key=cohere_api_key)
    chats["Cohere"] = chat_cohere
except:
    print("Could not initialize Cohere chatbot. Please store a valid API key in a .env file.")

# Llama 2 (Anyscale)
try: 
    anyscale_api_key = os.getenv("ANYSCALE_API_KEY")
    chat_llama = ChatAnyscale(model_name="meta-llama/Llama-2-7b-chat-hf", anyscale_api_key=anyscale_api_key)
    chats["Llama 2"] = chat_llama
except:
    print("Could not initialize Anyscale chatbot. Please store a valid API key in a .env file.")

## Experiments Set 1: All prompts, english language, all models

In [4]:
# Load the Miracl dataset (english version)
dataset = load_dataset("Cohere/miracl-en-queries-22-12", split="dev")
dataset = dataset.to_pandas().to_dict(orient='records')

# set up searcher
searcher = LuceneSearcher.from_prebuilt_index('miracl-v1.0-en')

## 1) Baseline

In [4]:
recall_baseline, ndcg_baseline = run_search(searcher, dataset)

print(f'BM25 Recall@100: {recall_baseline:.4f}')
print(f'BM25 nDCG@10: {ndcg_baseline:.4f}')
print("\n")

## 2) Query2Doc Zero Shot Prompting

In [6]:
for chat_name, chat in chats.items():
    expanded_dataset = get_query_expansion_dataset(dataset, chat, chat_name, lang='en', prompt='q2d-zs')

    recall, ndcg = run_search(searcher, expanded_dataset)
    print(f'{chat_name} Recall@100: {recall:.4f}') 
    print(f'{chat_name} nDCG@10: {ndcg:.4f}')
    print("\n")

Expanding queries:   0%|          | 0/5 [00:00<?, ?it/s]

Searching: 100%|██████████| 5/5 [00:14<00:00,  2.83s/it]


OpenAI Recall@100: 0.8333
OpenAI nDCG@10: 0.2921




Searching: 100%|██████████| 5/5 [00:17<00:00,  3.51s/it]


Cohere Recall@100: 0.7667
Cohere nDCG@10: 0.3049




Searching: 100%|██████████| 5/5 [00:08<00:00,  1.77s/it]

Llama 2 Recall@100: 0.7667
Llama 2 nDCG@10: 0.3563




## 3) Q2D zero shot prompting with pseudo-relevant feedback

In [ ]:
for chat_name, chat in chats.items():
    expanded_dataset = get_query_expansion_dataset(dataset, chat, chat_name, lang='en', prompt='q2d-zs-prf')

    recall, ndcg = run_search(searcher, expanded_dataset)
    print(f'{chat_name} Recall@100: {recall:.4f}') 
    print(f'{chat_name} nDCG@10: {ndcg:.4f}')
    print("\n")

Getting prf documents...


Searching: 100%|██████████| 5/5 [00:00<00:00, 72.71it/s]


Done.


Searching: 100%|██████████| 5/5 [00:08<00:00,  1.60s/it]

BM25 Recall@100: 0.9000
BM25 nDCG@10: 0.3924




## 4) Keywords with zero-shot prompting

In [8]:
for chat_name, chat in chats.items():
    expanded_dataset = get_query_expansion_dataset(dataset, chat, chat_name, lang='en', prompt='q2e-zs')

    recall, ndcg = run_search(searcher, expanded_dataset)
    print(f'{chat_name} Recall@100: {recall:.4f}') 
    print(f'{chat_name} nDCG@10: {ndcg:.4f}')
    print("\n")

Expanding queries:   0%|          | 0/799 [00:00<?, ?it/s]

Searching: 100%|██████████| 799/799 [01:25<00:00,  9.38it/s]

BM25 Recall@100: 0.8433
BM25 nDCG@10: 0.3745




## 5) Keywords with pseudo-relevant feedback

In [9]:
for chat_name, chat in chats.items():
    expanded_dataset = get_query_expansion_dataset(dataset, chat, chat_name, lang='en', prompt='q2e-zs-prf')

    recall, ndcg = run_search(searcher, expanded_dataset)
    print(f'{chat_name} Recall@100: {recall:.4f}') 
    print(f'{chat_name} nDCG@10: {ndcg:.4f}')
    print("\n")

Getting prf documents...


Searching:  11%|█         | 89/799 [00:02<00:28, 25.03it/s]

Searching: 100%|██████████| 799/799 [00:26<00:00, 30.04it/s]


Done.


Searching: 100%|██████████| 799/799 [02:56<00:00,  4.53it/s]

BM25 Recall@100: 0.8337
BM25 nDCG@10: 0.3722




## 6) Chain of thought prompting

In [15]:
for chat_name, chat in chats.items():
    expanded_dataset = get_query_expansion_dataset(dataset, chat, chat_name, lang='en', prompt='chain-of-thought')

    recall, ndcg = run_search(searcher, expanded_dataset)
    print(f'{chat_name} Recall@100: {recall:.4f}') 
    print(f'{chat_name} nDCG@10: {ndcg:.4f}')
    print("\n")

Expanding queries:  33%|███▎      | 1/3 [00:16<00:33, 16.59s/it]

Is Creole a pidgin of French? Is Creole a pidgin of French? Is Creole a pidgin of French? Is Creole a pidgin of French? Is Creole a pidgin of French? Yes, Creole can be considered a pidgin of French. 

A pidgin is a simplified language that develops as a means of communication between two groups of people who do not share a common language. It typically arises in situations of trade or colonization, where people with different native languages need to communicate.

Creole languages, including Haitian Creole and Louisiana Creole, emerged as pidgins when French-speaking colonizers interacted with African slaves and other non-French speakers. These pidgins gradually evolved and became the native languages of the communities, gaining complexity and distinct features over time.

Therefore, Creole languages can be seen as pidgins that have undergone further development and become fully-formed languages in their own right, while still maintaining influences from the French language.


Expanding queries:  67%|██████▋   | 2/3 [00:28<00:13, 13.94s/it]

What percentage of the Earth's atmosphere is oxygen? What percentage of the Earth's atmosphere is oxygen? What percentage of the Earth's atmosphere is oxygen? What percentage of the Earth's atmosphere is oxygen? What percentage of the Earth's atmosphere is oxygen? Approximately 21% of the Earth's atmosphere is oxygen. This percentage is based on the composition of the atmosphere which consists of various gases such as nitrogen, oxygen, carbon dioxide, and others. Oxygen is the second most abundant gas in the atmosphere after nitrogen. The rational behind this answer is that oxygen is essential for supporting life, and its presence in the atmosphere allows for respiration and combustion processes. However, it is important to note that the precise percentage may vary slightly depending on location and altitude.


Expanding queries: 100%|██████████| 3/3 [00:59<00:00, 19.76s/it]


...

What percentage of the Earth's atmosphere is oxygen? What percentage of the Earth's atmosphere is oxygen? What percentage of the Earth's atmosphere is oxygen? What percentage of the Earth's atmosphere is oxygen? What percentage of the Earth's atmosphere is oxygen?   The Earth's atmosphere is composed of several gases, with oxygen (O2) being the most abundant. According to the National Oceanic and Atmospheric Administration (NOAA), oxygen makes up approximately 21% of the Earth's atmosphere.

To calculate the percentage of oxygen in the atmosphere, we need to know the total volume of the atmosphere. The Earth's atmosphere is approximately 5.14 x 10^18 meters cubed, according to NASA.

Now, let's calculate the percentage of oxygen in the atmosphere:

Percentage of oxygen in the atmosphere = Mass of oxygen / Total mass of atmosphere

The mass of oxygen in the atmosphere is approximately 0.21 x 10^18 kilograms (based on the average atmospheric pressure of 1013 mbar and the molar mass of ox

Expanding queries: 100%|██████████| 3/3 [00:28<00:00,  9.47s/it]


When did Marxism develop? When did Marxism develop? When did Marxism develop? When did Marxism develop? When did Marxism develop?   Marxism, as a political and economic ideology, developed over a period of time, with key contributions from Karl Marx and Friedrich Engels in the 19th century. Here's a brief timeline of the major milestones in the development of Marxism:

1. Early influences (1815-1830s): Marx was influenced by the ideas of the French Revolution, the Enlightenment, and the young Hegelian school. He also read the works of Adam Smith, David Ricardo, and Thomas Malthus, which laid the groundwork for his ideas on capitalism and economic inequality.
2. Young Hegelian period (1830s-1840s): Marx was a member of the Young Hegelian circle in Berlin, where he developed his early ideas on dialectical materialism and the role of class struggle in history.
3. Parisian period (1843-1848): Marx moved to Paris, where he became familiar with the works of Charles Fourier, Louis Blanc, and 

Searching: 100%|██████████| 3/3 [00:09<00:00,  3.16s/it]

Llama 2 Recall@100: 0.6111
Llama 2 nDCG@10: 0.2592




## 7) Chain of thought prompting with psuedo-relevant feedback

In [7]:
for chat_name, chat in chats.items():
    expanded_dataset = get_query_expansion_dataset(dataset, chat, chat_name, lang='en', prompt='chain-of-thought-prf')

    recall, ndcg = run_search(searcher, expanded_dataset)
    print(f'{chat_name} Recall@100: {recall:.4f}') 
    print(f'{chat_name} nDCG@10: {ndcg:.4f}')
    print("\n")

Getting prf documents...


Searching: 100%|██████████| 799/799 [00:30<00:00, 25.81it/s]


Done.


Searching: 100%|██████████| 799/799 [09:45<00:00,  1.36it/s]

BM25 Recall@100: 0.8486
BM25 nDCG@10: 0.4343




## 8) Chain of thought prompting with short answer

In [5]:
for chat_name, chat in chats.items():
    expanded_dataset = get_query_expansion_dataset(dataset, chat, chat_name, lang='en', prompt='chain-of-thought-short')

    recall, ndcg = run_search(searcher, expanded_dataset)
    print(f'{chat_name} Recall@100: {recall:.4f}') 
    print(f'{chat_name} nDCG@10: {ndcg:.4f}')
    print("\n")

# openai time: 2:04:00
# cohere time: 
# llama2 time: 

Searching:   0%|          | 0/799 [00:00<?, ?it/s]

Searching:  33%|███▎      | 262/799 [03:28<05:23,  1.66it/s]

# Experiments Set 2: Chain-of-thought prompt, various languages, all models

## 1) French

In [ ]:
# Load the Miracl dataset (english version)
dataset = load_dataset("Cohere/miracl-fr-queries-22-12", split="dev")
dataset = dataset.to_pandas().to_dict(orient='records')

# set up searcher
searcher = LuceneSearcher.from_prebuilt_index('miracl-v1.0-fr')
searcher.set_language('fr')

# exclude cohere model as it does not support french
chats_fr = {k: v for k, v in chats.items() if k != "Cohere"}

In [ ]:
recall_baseline, ndcg_baseline = run_search(searcher, dataset)

print(f'BM25 Recall@100: {recall_baseline:.4f}')
print(f'BM25 nDCG@10: {ndcg_baseline:.4f}')
print("\n")

Searching: 100%|██████████| 343/343 [00:11<00:00, 29.70it/s]

BM25 Recall@100: 0.6528
BM25 nDCG@10: 0.1832




In [ ]:
for chat_name, chat in chats_fr.items():
    expanded_dataset = get_query_expansion_dataset(dataset, chat, chat_name, lang='fr', prompt='chain-of-thought-fr')

    recall, ndcg = run_search(searcher, expanded_dataset)
    print(f'{chat_name} Recall@100: {recall:.4f}') 
    print(f'{chat_name} nDCG@10: {ndcg:.4f}')
    print("\n")

# openai time: 
# llama 2 time: 25:12 min

Searching:   0%|          | 0/343 [00:00<?, ?it/s]

Searching: 100%|██████████| 343/343 [01:09<00:00,  4.92it/s]


Llama 2 Recall@100: 0.3979
Llama 2 nDCG@10: 0.1341




Expanding queries:   4%|▍         | 14/343 [17:39<6:55:00, 75.68s/it]


KeyboardInterrupt: 

## 2) German

In [ ]:
# Load the Miracl dataset (english version)
dataset = load_dataset("Cohere/miracl-de-queries-22-12", split="dev")
dataset = dataset.to_pandas().to_dict(orient='records')

# set up searcher
searcher = LuceneSearcher.from_prebuilt_index('miracl-v1.0-de')
searcher.set_language('de')

# exclude cohere model as it does not support french
chats_de = {k: v for k, v in chats.items() if k != "Cohere"}

In [ ]:
recall_baseline, ndcg_baseline = run_search(searcher, dataset)

print(f'BM25 Recall@100: {recall_baseline:.4f}')
print(f'BM25 nDCG@10: {ndcg_baseline:.4f}')
print("\n")

Searching: 100%|██████████| 801/801 [00:15<00:00, 50.50it/s]

BM25 Recall@100: 0.0000
BM25 nDCG@10: 0.0000




In [ ]:
for chat_name, chat in chats_de.items():
    expanded_dataset = get_query_expansion_dataset(dataset[:10], chat, chat_name, lang='de', prompt='chain-of-thought-de', verbose=True)

    recall, ndcg = run_search(searcher, expanded_dataset)
    print(f'{chat_name} Recall@100: {recall:.4f}') 
    print(f'{chat_name} nDCG@10: {ndcg:.4f}')
    print("\n")

# openai time: 
# llama 2 time: 25:12 min

Expanding queries:   0%|          | 0/10 [00:00<?, ?it/s]

Expanding queries:  10%|█         | 1/10 [00:05<00:48,  5.43s/it]

二战是什么时候开始的？ 二战是什么时候开始的？ 二战是什么时候开始的？ 二战是什么时候开始的？ 二战是什么时候开始的？ 二战始于1939年。这场全球战争爆发的导火索是德国入侵波兰，引发了英法对德宣战。


Expanding queries:  20%|██        | 2/10 [00:12<00:49,  6.20s/it]

复旦大学在哪个城市？ 复旦大学在哪个城市？ 复旦大学在哪个城市？ 复旦大学在哪个城市？ 复旦大学在哪个城市？ 复旦大学位于中国上海市。上海是中国的经济中心和国际大都市，拥有丰富的资源和机遇，吸引了许多学生和学者。


...

印度人口有多少？ 印度人口有多少？ 印度人口有多少？ 印度人口有多少？ 印度人口有多少？   Sure! Here's the answer to your question:

Question: How many people live in India?

Reason: India is a vast and populous country with a diverse population.

Answer: According to the latest estimates, the population of India is approximately 1.38 billion people. (2020 figures)

Note: The population of India is constantly growing due to a high birth rate and relatively low death rate, as well as internal migration and urbanization.


Expanding queries: 100%|██████████| 10/10 [00:23<00:00,  2.32s/it]


电视剧《继承者们》的女主角是谁？ 电视剧《继承者们》的女主角是谁？ 电视剧《继承者们》的女主角是谁？ 电视剧《继承者们》的女主角是谁？ 电视剧《继承者们》的女主角是谁？   Sure, I'd be happy to help!

The female lead in the TV drama "继承者们" is Qin Shuang.

Reason: Qin Shuang is the central character of the drama and the story revolves around her journey as a young heiress navigating the complexities of her family's business and personal relationships. She is portrayed by actress Zhang Zhehan.


Searching: 100%|██████████| 10/10 [00:00<00:00, 14.47it/s]

Llama 2 Recall@100: 0.0000
Llama 2 nDCG@10: 0.0000


